# Exploração e pré-processamento dos dados

In [ ]:
!pip install pandas matplotlib seaborn

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar o dataset
df = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")

# Adicionar rótulo
df['label'] = 0  # Fake
df_real['label'] = 1  # Real

# Unir os dois
df_all = pd.concat([df, df_real])
df_all = df_all.sample(frac=1).reset_index(drop=True)

# Ver primeiras linhas
df_all.head()

,title,text,subject,date,label
0,Greedy Trump Cuts Takes Housing Money Away Fr...,Donald Trump s budget is all about punishing t...,News,"June 21, 2017",0
1,Insight: Rap and the Party: China taps youth c...,BEIJING (Reuters) - In his baseball cap and b...,worldnews,"October 16, 2017",1
2,"'Political mainstream', Corbyn says Britain's ...","BRIGHTON, England (Reuters) - Opposition leade...",worldnews,"September 26, 2017",1
3,'Lips and teeth' no more as China's ties with ...,BEIJING (Reuters) - When Kim Jong Un inherited...,worldnews,"September 8, 2017",1
4,WATCH: Dr. Ben Carson Explains Why He’s Endors...,There are two different Donald Trump s Here s...,politics,"Mar 11, 2016",0


In [ ]:
sns.countplot(x='label', data=df_all)
plt.title("Distribuição de Notícias Falsas e Reais")
plt.xticks([0,1], ['Fake', 'Real'])
plt.show()

## Pré-processamento de texto (PLN)

limpeza dos dados

In [4]:
import re # expressões regulares
import nltk # usada para processamento de linguagem natural
from nltk.corpus import stopwords # conjunto de palavras irrelevantes (stopwords) da NLTK
from nltk.stem import PorterStemmer # reduz as palavras ao seu radical

nltk.download('stopwords')

# Cria um conjunto com as stopwords em inglês
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = re.sub(r'\W', ' ', text) # substitui qualquer caractere que não seja uma letra ou número por espaço

    text = text.lower()
    text = text.split()

    text = [stemmer.stem(word) for word in text if word not in stop_words] # remove stopwords e aplica stemming em cada palavra

    return ' '.join(text)

df_all['text_clean'] = df_all['title'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Vetorização

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_all['text_clean']).toarray()
y = df_all['label']

Divisão dos dados

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Algoritmo 1: Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

start = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=15) # aqui da pra tester mais arvores de decisao (talvez testar 1000?)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_pred_train_rf = rf.predict(X_train)
end = time.time()

print("Acurácia do conjunto de treino:", accuracy_score(y_train, y_pred_train_rf))
print("Acurácia do conjunto de teste:", accuracy_score(y_test, y_pred_rf))
print("Tempo de execução:", round(end - start, 2), "s")
print(classification_report(y_test, y_pred_rf))

Acurácia do conjunto de treino: 0.8986580544573751
Acurácia do conjunto de teste: 0.8847438752783965
Tempo de execução: 34.99 s
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      4648
           1       0.88      0.88      0.88      4332

    accuracy                           0.88      8980
   macro avg       0.88      0.88      0.88      8980
weighted avg       0.88      0.88      0.88      8980



## Algoritmo 2:  Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

start = time.time()
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
y_pred_train_nb = nb.predict(X_train)
end = time.time()

print("Acurácia do conjunto de treino:", accuracy_score(y_train, y_pred_train_nb))
print("Acurácia do conjunto de teste:", accuracy_score(y_test, y_pred_nb))
print("Tempo de execução:", round(end - start, 2), "s")
print(classification_report(y_test, y_pred_nb))


## Gráfico comparando os dois algoritmos

In [ ]:
resultados_reais = pd.DataFrame({
    'Algoritmo': ['Random Forest (n=100)', 'Naive Bayes'],
    'Acurácia': [0.940, 0.925],
    'Precisão': [0.94, 0.93],
    'Recall': [0.94, 0.92],
    'F1-Score': [0.94, 0.92],
    'Tempo Execução (s)': [281.48, 0.9]
})

# Criar gráfico de comparação de métricas reais
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
metrics = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']

for ax, metric in zip(axs.ravel(), metrics):
    ax.bar(resultados_reais['Algoritmo'], resultados_reais[metric], color=['#1f77b4', '#ff7f0e'])
    ax.set_title(metric)
    ax.set_ylim(0.90, 0.96)
    ax.set_ylabel(metric)
    ax.set_xticklabels(resultados_reais['Algoritmo'], rotation=15, ha='right')

plt.tight_layout()

# Tabela também
resultados_reais.sort_values(by='Acurácia', ascending=False), plt.show()


## Algoritmo 3: Support Vector Machine SVM

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

start = time.time()

# cria o modelo SVM com kernel linear pq e mais rápido e apropriado para texto)
svm = LinearSVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
y_pred_train_svm = svm.predict(X_train)
end = time.time()

print("Acurácia do conjunto de treino:", accuracy_score(y_train, y_pred_train_svm))
print("Acurácia do conjunto de teste:", accuracy_score(y_test, y_pred_svm))
print("Tempo de execução:", round(end - start, 2), "s")
print("Relatório:\n", classification_report(y_test, y_pred_svm))

## Algoritmo 4: KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

start = time.time()

# cria o modelo com K=5 vizinhos
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_pred_train_knn = knn.predict(X_train)
end = time.time()

print("Acurácia do conjunto de treino:", accuracy_score(y_train, y_pred_train_knn))
print("Acurácia do conjunto de teste:", accuracy_score(y_test, y_pred_knn))
print("Tempo de execução:", round(end - start, 2), "s")
print("Relatório:\n", classification_report(y_test, y_pred_knn))

## Análise resultados

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

modelos = ['Random Forest', 'Naive Bayes', 'SVM', 'KNN']

acuracia = [0.885, 0.930, 0.947, 0.890]
precisao = [0.89, 0.93, 0.95, 0.89]  # Média ponderada dos valores de precisão
recall = [0.89, 0.93, 0.95, 0.89]    # Média ponderada dos valores de recall
f1_score = [0.88, 0.93, 0.95, 0.89]  # Média ponderada dos valores de F1-score
tempo_execucao = [40.83, 1.33, 1.34, 700.43]

plt.figure(figsize=(15, 10))

# Gráfico 1 - Acurácia
plt.subplot(2, 2, 1)
bars = plt.bar(modelos, acuracia, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Comparação de Acurácia entre Modelos (Conjunto de Teste)')
plt.ylabel('Acurácia')
plt.ylim(0.8, 1.0)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}', ha='center', va='bottom')

# Gráfico 2 - Precisão
plt.subplot(2, 2, 2)
bars = plt.bar(modelos, precisao, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Comparação de Precisão entre Modelos (Média Ponderada)')
plt.ylabel('Precisão')
plt.ylim(0.8, 1.0)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}', ha='center', va='bottom')

# Gráfico 3 - Recall
plt.subplot(2, 2, 3)
bars = plt.bar(modelos, recall, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Comparação de Recall entre Modelos (Média Ponderada)')
plt.ylabel('Recall')
plt.ylim(0.8, 1.0)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}', ha='center', va='bottom')

# Gráfico 4 - F1-Score
plt.subplot(2, 2, 4)
bars = plt.bar(modelos, f1_score, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Comparação de F1-Score entre Modelos (Média Ponderada)')
plt.ylabel('F1-Score')
plt.ylim(0.8, 1.0)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Gráfico 5 - Tempo de Execução (escala logarítmica devido ao KNN)
plt.figure(figsize=(8, 5))
bars = plt.bar(modelos, tempo_execucao, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Comparação de Tempo de Execução entre Modelos')
plt.ylabel('Tempo (segundos) - Escala Logarítmica')
plt.yscale('log')  # Escala logarítmica para melhor visualização
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}s', ha='center', va='bottom')
plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

modelos = {
    "Random Forest": y_pred_rf,
    "Naive Bayes": y_pred_nb,
    "Logistic Regression": y_pred_lr,
    "SVM": y_pred_svm
}

# Plota matriz de confusão para cada modelo
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
axs = axs.flatten()

for i, (nome, pred) in enumerate(modelos.items()):
    disp = ConfusionMatrixDisplay.from_predictions(y_test, pred, ax=axs[i], cmap="Blues")
    axs[i].set_title(nome)

plt.tight_layout()
plt.show()
